In [1]:
# This notebook tests the new function calling feature of GPT-3.5-turbo.

# Set up chatgpt api
import openai
import json
import os

In [2]:
GPT_KEY = os.environ['OPENAI_API_KEY']
GPT_MODEL_OLD = "gpt-3.5-turbo"
GPT_MODEL = "gpt-3.5-turbo-0613"
GPT4_MODEL = "gpt-4-0613"


In [3]:
import sqlite3
from pprint import pprint

In [124]:
# We can choose to expose only actual results and hide the function calls
# Also, we can add some color coding to the output
def pprint_convo(convo, hide_internal=True):
    CYAN, GREEN, RED, BOLD = "\033[96m", "\033[92m", "\033[31m", "\033[1m"
    col_role = {"user":CYAN, "assistant":GREEN, "system":RED, "function":RED}
    begin_role = {"user":"\n", "assistant":"", "system":"", "function":""}
    
    for mes in convo:
        role = mes["role"]
        cont = mes["content"]
        if role not in ["user", "assistant"] or mes.get("function_call"):
            if hide_internal:
                continue
        if cont is not None:
            print("{fm_role}{role}:\033[0m {fm_content}{cont}\033[0m".format(
                fm_role = begin_role[role] + col_role[role] + BOLD,
                role=role,
                fm_content = col_role[role],
                cont=cont
            ))
        else:
            fc_content = json.loads(mes.get("function_call")["arguments"])
            print("""{fm_role}{role} function:\033[0m {fm_content}""".format(
                fm_role = begin_role["function"] + col_role["function"] + BOLD,
                role=role,
                fm_content = col_role["function"]
            ))
            pprint(fc_content)
            print("\033[0m")


## Clarification questions

In [6]:
# https://github.com/MeiqiGuo/AKBC2021-Abg-CoQA/tree/main/abg-coqa
with open("./data/abg-coqa/coqa_abg_test.json") as f:
    data_coqa = json.load(f)['data']


In [7]:
len(data_coqa)

1055

In [8]:
story2 = data_coqa[2]
story2.keys()

dict_keys(['id', 'story', 'target_turn', 'history_turns', 'ambiguity', 'clarification_turn', 'source', 'clarification_turn_2'])

In [9]:
pprint(story2['story'])

('Angie went to the library with her mother. First she had to turn in the '
 'books she was returning at the return desk. They said hello to the man '
 'there. He took their books. Then they went into the adult reading room. '
 'Angie sat in a brown chair at the table. She made a drawing of her mother. '
 'Her mother found a large red book. Then they went to the Mystery section. '
 'Angie sat in a blue chair. She drew a picture of her brother. Her mother '
 'found the book. It was a green book. Finally it was time to go to the '
 "children's room. It was Story Hour. Miss Hudson was there to read to all the "
 'children. She read a book about friendship. After the story Angie sat in the '
 'red chair and began drawing. They were drawing pictures of friends. Angie '
 'drew a picture of her best friend Lilly. Miss Hudson hung the pictures on '
 'the wall. Then Angie and her mother picked out 8 books to read at home. They '
 'checked the books out and went home.')


In [10]:
story2['history_turns']

[{'turn_id': 3,
  'question': 'what did she draw?',
  'answer': 'her mother',
  'rationale': '. She made a drawing of her mother. '},
 {'turn_id': 4,
  'question': 'what did her mother find?',
  'answer': 'the book.\\',
  'rationale': ' Her mother found the book.'}]

In [11]:
story2['target_turn']

{'turn_id': 5,
 'question': 'what color was it?',
 'answer': 'green',
 'rationale': ' It was a green book.',
 'span_start': 452,
 'span_end': 473}

In [12]:
story2['clarification_turn']

{'question': 'Do you mean the first book?',
 'answers': [{'clr_ans': 'Yes',
   'org_ans': 'red',
   'org_ans_2': 'red',
   'org_ans_3': 'red'},
  {'clr_ans': 'No, I mean the second book.',
   'org_ans': 'green',
   'org_ans_2': 'green',
   'org_ans_3': 'green'}]}

In [13]:
story2['clarification_turn_2']

{'question': 'Do you mean the book?',
 'answers': [{'clr_ans': 'Yes', 'org_ans': 'red'},
  {'clr_ans': 'No, I mean the drawing of her mother.', 'org_ans': 'Unknown'}]}

### Input-Output

In [75]:
def cq_generate_convo_io(convo, max_internal_turns=10, gpt_model=GPT_MODEL):
    i = 0
    while ((i <= max_internal_turns) and not (
        convo[-1]["role"] == "assistant" and not convo[-1].get("function_call")
        )):
        # Get gpt response
        gpt_response = openai.ChatCompletion.create(
            model=gpt_model,
            messages=convo,
        )
        # Extract the assistant's message and append to the conversation
        assistant_message = gpt_response["choices"][0]["message"]
        convo.append(assistant_message)
        i += 1
    return convo


In [83]:

convo = []
convo.append({"role": "system", "content": f"""Answer user questions by 
    referring to the story. The story: \n{story2['story']}"""})
# convo.append({"role":"system", "content":"The story:\n" + story2['story']})
# convo.append({"role": "user", "content": "Hi, how many apples were sold?"})
for turn in story2["history_turns"]:
    convo.append({"role":"user", "content": turn["question"]})
    convo.append({"role":"assistant", "content": turn["answer"]})
convo.append({"role":"user", "content": story2["target_turn"]["question"]})
pprint_convo(convo=convo, hide_internal=False)

system: Answer user questions by 
    referring to the story. The story: 
Angie went to the library with her mother. First she had to turn in the books she was returning at the return desk. They said hello to the man there. He took their books. Then they went into the adult reading room. Angie sat in a brown chair at the table. She made a drawing of her mother. Her mother found a large red book. Then they went to the Mystery section. Angie sat in a blue chair. She drew a picture of her brother. Her mother found the book. It was a green book. Finally it was time to go to the children's room. It was Story Hour. Miss Hudson was there to read to all the children. She read a book about friendship. After the story Angie sat in the red chair and began drawing. They were drawing pictures of friends. Angie drew a picture of her best friend Lilly. Miss Hudson hung the pictures on the wall. Then Angie and her mother picked out 8 books to read at home. They checked the books out and went home.

us

In [84]:
cq_generate_convo_io(convo=convo, gpt_model=GPT_MODEL)
pprint_convo(convo=convo, hide_internal=False)

system: Answer user questions by 
    referring to the story. The story: 
Angie went to the library with her mother. First she had to turn in the books she was returning at the return desk. They said hello to the man there. He took their books. Then they went into the adult reading room. Angie sat in a brown chair at the table. She made a drawing of her mother. Her mother found a large red book. Then they went to the Mystery section. Angie sat in a blue chair. She drew a picture of her brother. Her mother found the book. It was a green book. Finally it was time to go to the children's room. It was Story Hour. Miss Hudson was there to read to all the children. She read a book about friendship. After the story Angie sat in the red chair and began drawing. They were drawing pictures of friends. Angie drew a picture of her best friend Lilly. Miss Hudson hung the pictures on the wall. Then Angie and her mother picked out 8 books to read at home. They checked the books out and went home.

us

### Proactive CoT

In [111]:
desc__clarify = {
    "name": "clarify",
    "description": """Given the task background and the conversation 
                history, first analyze whether the question at the current turn
                is ambiguous to answer, then select appropriate actions to 
                generate the response""",
    "parameters":{
        "type": "object",
        "properties": {
            "analysis":{
                "type": "string",
                "description": """Analysis if the question at the current turn
                is ambiguous to answer, the result of this analysis should be
                the rationale for the action."""
            },
            "action":{
                "type": "string",
                "description": """Appropriate action to take based on the analysis""",
                "enum": ["Direct answer", "Ask a Clarification Question"]
            },
            "clarification_question":{
                "type": "string",
                "description": """Clarification to ask the user if there was 
                ambiguity, it should be short and concise"""
            }
        },
        "required": ["analysis", "action"],
    }
}

In [132]:
def cq_generate_convo_cot(
        convo, max_internal_turns=10, gpt_model=GPT_MODEL, 
        force_fc=True):
    i = 0
    while ((i <= max_internal_turns) and not (
        convo[-1]["role"] == "assistant" and not convo[-1].get("function_call")
        )):
        if convo[-1].get("role") == "user" and force_fc:
            fc = {"name": "clarify"}
        else:
            fc = "auto"
        # Get gpt response
        gpt_response = openai.ChatCompletion.create(
            model=gpt_model,
            messages=convo,
            functions=[desc__clarify],
            function_call=fc,
        )
        # Extract the assistant's message and append to the conversation
        assistant_message = gpt_response["choices"][0]["message"]
        convo.append(assistant_message)
        i += 1
    return convo


In [133]:

convo = []
convo.append({"role": "system", "content": f"""Answer user questions by 
    referring to the story. The story: \n{story2['story']}"""})
# convo.append({"role":"system", "content":"The story:\n" + story2['story']})\
for turn in story2["history_turns"]:
    convo.append({"role":"user", "content": turn["question"]})
    convo.append({"role":"assistant", "content": turn["answer"]})
convo.append({"role":"user", "content": story2["target_turn"]["question"]})
pprint_convo(convo=convo, hide_internal=False)

system: Answer user questions by 
    referring to the story. The story: 
Angie went to the library with her mother. First she had to turn in the books she was returning at the return desk. They said hello to the man there. He took their books. Then they went into the adult reading room. Angie sat in a brown chair at the table. She made a drawing of her mother. Her mother found a large red book. Then they went to the Mystery section. Angie sat in a blue chair. She drew a picture of her brother. Her mother found the book. It was a green book. Finally it was time to go to the children's room. It was Story Hour. Miss Hudson was there to read to all the children. She read a book about friendship. After the story Angie sat in the red chair and began drawing. They were drawing pictures of friends. Angie drew a picture of her best friend Lilly. Miss Hudson hung the pictures on the wall. Then Angie and her mother picked out 8 books to read at home. They checked the books out and went home.

us

In [134]:
cq_generate_convo_cot(convo=convo, max_internal_turns=10, gpt_model=GPT_MODEL,
                      force_fc=True)
pprint_convo(convo=convo, hide_internal=False)

system: Answer user questions by 
    referring to the story. The story: 
Angie went to the library with her mother. First she had to turn in the books she was returning at the return desk. They said hello to the man there. He took their books. Then they went into the adult reading room. Angie sat in a brown chair at the table. She made a drawing of her mother. Her mother found a large red book. Then they went to the Mystery section. Angie sat in a blue chair. She drew a picture of her brother. Her mother found the book. It was a green book. Finally it was time to go to the children's room. It was Story Hour. Miss Hudson was there to read to all the children. She read a book about friendship. After the story Angie sat in the red chair and began drawing. They were drawing pictures of friends. Angie drew a picture of her best friend Lilly. Miss Hudson hung the pictures on the wall. Then Angie and her mother picked out 8 books to read at home. They checked the books out and went home.

us

In [135]:
convo.append({
    "role": "user", 
    "content": "The second one"
    })
cq_generate_convo_cot(convo)
pprint_convo(convo, hide_internal=False)

system: Answer user questions by 
    referring to the story. The story: 
Angie went to the library with her mother. First she had to turn in the books she was returning at the return desk. They said hello to the man there. He took their books. Then they went into the adult reading room. Angie sat in a brown chair at the table. She made a drawing of her mother. Her mother found a large red book. Then they went to the Mystery section. Angie sat in a blue chair. She drew a picture of her brother. Her mother found the book. It was a green book. Finally it was time to go to the children's room. It was Story Hour. Miss Hudson was there to read to all the children. She read a book about friendship. After the story Angie sat in the red chair and began drawing. They were drawing pictures of friends. Angie drew a picture of her best friend Lilly. Miss Hudson hung the pictures on the wall. Then Angie and her mother picked out 8 books to read at home. They checked the books out and went home.

us

## Database query

In [31]:


conn = sqlite3.connect('data/sqlite/grocery_txn_2.db')
conn.execute("""CREATE TABLE IF NOT EXISTS transactions (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_name TEXT NOT NULL,
    product_name TEXT NOT NULL,
    product_category TEXT NOT NULL,
    quantity INTEGER NOT NULL,
    price REAL NOT NULL,
    total REAL NOT NULL
);""")
rows = [
    ('john', 'apple', 'fruit', 10, 1.0, 10.0),
    ('john', 'orange', 'fruit', 5, 0.5, 2.5),
    ('john', 'spaghetti', 'dry', 3, 1.2, 3.6),
    ('john', 'chicken', 'meat', 3, 1, 3),
    ('hannah', 'apple', 'fruit', 5, 1.0, 5.0),
    ('hannah', 'orange', 'fruit', 10, 0.5, 5.0),
    ('hannah', 'banana', 'fruit', 5, 0.2, 1), 
    ('hannah', 'rice', 'dry', 5, 0.8, 4),
    ('alex', 'pork', 'meat', 2, 3, 6),
    ('alex', 'orange', 'fruit', 2, 0.5, 1),]
conn.executemany("""
    INSERT INTO transactions (user_name, product_name, product_category, quantity, price, total)
    VALUES (?, ?, ?, ?, ?, ?)""", 
    rows)
conn.commit()
print("id, user_name, product_name, product_category, quantity, price, total")
pprint(conn.execute("SELECT * FROM transactions").fetchall())
conn.execute(
    "SELECT user_name, sum(total) as spending FROM transactions group by 1"
    ).fetchall()

database_schema_string = """Table: transactions
    Columns: id, user_name, product_name, product_category, quantity, price, total"""

id, user_name, product_name, product_category, quantity, price, total
[(1, 'john', 'apple', 'fruit', 10, 1.0, 10.0),
 (2, 'john', 'orange', 'fruit', 5, 0.5, 2.5),
 (3, 'john', 'spaghetti', 'dry', 3, 1.2, 3.6),
 (4, 'john', 'chicken', 'meat', 3, 1.0, 3.0),
 (5, 'hannah', 'apple', 'fruit', 5, 1.0, 5.0),
 (6, 'hannah', 'orange', 'fruit', 10, 0.5, 5.0),
 (7, 'hannah', 'banana', 'fruit', 5, 0.2, 1.0),
 (8, 'hannah', 'rice', 'dry', 5, 0.8, 4.0),
 (9, 'alex', 'pork', 'meat', 2, 3.0, 6.0),
 (10, 'alex', 'orange', 'fruit', 2, 0.5, 1.0),
 (11, 'john', 'apple', 'fruit', 10, 1.0, 10.0),
 (12, 'john', 'orange', 'fruit', 5, 0.5, 2.5),
 (13, 'john', 'spaghetti', 'dry', 3, 1.2, 3.6),
 (14, 'john', 'chicken', 'meat', 3, 1.0, 3.0),
 (15, 'hannah', 'apple', 'fruit', 5, 1.0, 5.0),
 (16, 'hannah', 'orange', 'fruit', 10, 0.5, 5.0),
 (17, 'hannah', 'banana', 'fruit', 5, 0.2, 1.0),
 (18, 'hannah', 'rice', 'dry', 5, 0.8, 4.0),
 (19, 'alex', 'pork', 'meat', 2, 3.0, 6.0),
 (20, 'alex', 'orange', 'fruit', 2, 0.5,

In [32]:
desc__query_database = {
    "name": "query_database",
    "description": "Generate fully-formed SQL query to answer user's questions",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": f"""The SQL query extracting relevant info to 
                    the user's question. Use the following schema:
                        {database_schema_string}""",
            }
        },
        "required": ["query"],
    },
}

def query_database(conn, query):
    try:
        results = str(conn.execute(query).fetchall())
    except Exception as e:
        results = f"Query execution failed with error: {e}"
    return results

def execute_function_call(f_call):
    if f_call["name"] == "query_database":
        q = json.loads(f_call["arguments"])["query"]
        results = query_database(conn, q)
    else:
        results = f"""Error: function {f_call["name"]} does not exist"""
    return results

In [33]:
def generate_convo(convo, max_internal_turns=10):
    i = 0
    while ((i <= max_internal_turns) and not (
        convo[-1]["role"] == "assistant" and not convo[-1].get("function_call")
        )):
        # Get gpt response
        gpt_response = openai.ChatCompletion.create(
            model=GPT_MODEL,
            messages=convo,
            functions=[desc__query_database],
        function_call="auto",
        )
        # Extract the assistant's message and append to the conversation
        assistant_message = gpt_response["choices"][0]["message"]
        convo.append(assistant_message)
        # If the model suggested using a function call, get the result
        f_call = assistant_message.get("function_call")
        if f_call:
            results = execute_function_call(f_call)
            convo.append({
                "role": "function", "name": f_call["name"], "content": results
                })
        i += 1
    return convo


In [34]:
convo = []
convo.append({"role": "system", "content": """Answer user questions by 
    understanding the user's request, asking clarification questions, and 
    generating SQL queries against the Grocery Transaction Database
    if additional data is needed."""})
convo.append({"role": "user", "content": "Hi, please report the current business situation"})

In [35]:
generate_convo(convo)
pprint_convo(convo, hide_internal=True)


user: Hi, please report the current business situation
assistant: Sure, I need to access the Grocery Transaction Database to retrieve the necessary information. Could you please provide the SQL query that I should use to extract the relevant data for reporting the current business situation?


In [36]:
convo.append({
    "role": "user", 
    "content": "I want to know what products I should buy order from our suppliers"
    })
generate_convo(convo)
pprint_convo(convo, hide_internal=False)


system: Answer user questions by 
    understanding the user's request, asking clarification questions, and 
    generating SQL queries against the Grocery Transaction Database
    if additional data is needed.

user: Hi, please report the current business situation
assistant: Sure, I need to access the Grocery Transaction Database to retrieve the necessary information. Could you please provide the SQL query that I should use to extract the relevant data for reporting the current business situation?

user: I want to know what products I should buy order from our suppliers
assistant: None
function: [('pork', 4), ('chicken', 6), ('spaghetti', 6), ('banana', 10), ('rice', 10), ('apple', 30), ('orange', 34)]
assistant: Based on the current sales data from the Grocery Transaction Database, here are the products that you should consider ordering from your suppliers in ascending order of total quantity sold:

1. Pork - 4 units sold
2. Chicken - 6 units sold
3. Spaghetti - 6 units sold
4. Bana